## Проверка модулей

In [1]:
import pandas as pd
import numpy as np

from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items, top_items
from src.recommenders import MainRecommender

In [2]:
data = pd.read_csv('transaction_data.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

#data_train = prefilter_items(data_train)
data_train_top = top_items(data_train) # top 5000 товаров

data_train[:2]

C:\DS_Soft\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65..."
1,3,"[823704, 834117, 840244, 913785, 917816, 93870..."


In [4]:
recoms = MainRecommender(data_train_top)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

**ALS**

In [8]:
result['bm25'] = result['user_id'].apply(lambda x: recoms.get_recommendation(x, N=5))

In [10]:
score = {}
score['bm25_precision'] = result.apply(lambda row: precision_at_k(row['bm25'], row['actual']), axis=1).mean()

**Похожие товары**

In [11]:
result['similar_rec'] =  result['user_id'].apply(lambda x: recoms.get_similar_items_recommendation(data_train_top, x, N=5))

In [12]:
score['similar_rec_precision'] = result.apply(lambda row: precision_at_k(row['similar_rec'], row['actual']), axis=1).mean()

F:\DataSc\GeekBr\Rec_Sys\HWs\HW_4\src\metrics.py:17: RuntimeWarning: invalid value encountered in long_scalars
  precision = flags.sum() / len(recommended_list)


**Похожим пользователям нравится**

In [13]:
result['similar_users'] =  result['user_id'].apply(lambda x: recoms.get_similar_users_recommendation(x, N=5))

In [14]:
score['similar_users'] = result.apply(lambda row: precision_at_k(row['similar_users'], row['actual']), axis=1).mean()

**Оценка качества**

In [23]:
score = pd.DataFrame(score.items())
score

,0,1
0,bm25_precision,0.120542
1,similar_rec_precision,0.091478
2,similar_users,0.004370


**Результаты на top 10000 товаров**

In [17]:
data_train_top = top_items(data_train, top_num=10000) # top 10000 товаров

C:\DS_Soft\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [18]:
recoms = MainRecommender(data_train_top)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

In [19]:
result['bm25'] = result['user_id'].apply(lambda x: recoms.get_recommendation(x, N=5))
result['similar_rec'] =  result['user_id'].apply(lambda x: recoms.get_similar_items_recommendation(data_train_top, x, N=5))
result['similar_users'] =  result['user_id'].apply(lambda x: recoms.get_similar_users_recommendation(x, N=5))

In [20]:
score_2 = {}
score_2['bm25_precision'] = result.apply(lambda row: precision_at_k(row['bm25'], row['actual']), axis=1).mean()
score_2['similar_rec_precision'] = result.apply(lambda row: precision_at_k(row['similar_rec'], row['actual']), axis=1).mean()
score_2['similar_users'] = result.apply(lambda row: precision_at_k(row['similar_users'], row['actual']), axis=1).mean()

F:\DataSc\GeekBr\Rec_Sys\HWs\HW_4\src\metrics.py:17: RuntimeWarning: invalid value encountered in long_scalars
  precision = flags.sum() / len(recommended_list)


**Сравнение оценок качества**

In [25]:
score = score.merge(pd.DataFrame(score_2.items()), on=0, how='left')
score

,0,1_x,1_y
0,bm25_precision,0.120542,0.123757
1,similar_rec_precision,0.091478,0.086482
2,similar_users,0.004370,0.004671
